In [90]:
import pandas as pd
from neo4j import GraphDatabase
import pymongo

In [88]:
#Neo4j
neo_connection = GraphDatabase.driver(
    uri = "bolt://localhost:7687", auth=("neo4j", "neo5j"))
session = neo_connection.session()    

In [92]:
#mongo
client=pymongo.MongoClient('mongodb://127.0.0.1:27017/')
mydb = client['Practica4']
table = mydb["tabla_peliculas"]

In [50]:
print(string[:50])
print(string[51:68])
print(string[68:85])
print(string[85:115])
print(string[115:])

MATCH (a1:actor {nombre:'George \'Gabby\' Hayes'})
MATCH (p:titulo) 
MATCH (a2:actor) 
MATCH (a1)-[algo]->(p)-->(a2) 
RETURN a1,a2,p


In [28]:
name = r'George \'Gabby\' Hayes'
c0 = f"match (n:persona {{nombre:'{name}'}}) \
match (t:titulo) \
match (n) -[r]- (t) \
return r, t"
c0

"match (n:persona {nombre:'George \\'Gabby\\' Hayes'}) match (t:titulo) match (n) -[r]- (t) return r, t"

In [53]:
#actores que trabajaron juntos
ac = ['Si Jenks', 'Hal Price']

#Dados dos actores, obtener peliculas en las que trabajaron juntos.

match (a:actor {nombre: _actorA_})
match (b:actor {nombre: _actorB_})
match (t:titulo)
match (a) -- t -- (b)
return t

c0 = f"\
match (a:actor {{nombre: '{ac[0]}'}}) \
match (b:actor {{nombre: '{ac[1]}'}}) \
match (t:titulo) \
match (a) -- (t) -- (b) \
return (t)\
"
c0

In [85]:
session.run(c0).data()[0]['t']

{'id_titulo': 'tm132612', 'nombre': 'Riders of Destiny'}

In [ ]:
#Dado un director, encuentra actores que fueron dirigidos por él.

match (a:actor)
match (d:director {nombre: "_nombredir_"})
match (t:titulo)
match (d) -- (t) -- (a)
return a;

In [72]:
director = 'Wallace Fox'

c1 = f"\
match (a:actor) \
match (d:director {{nombre: '{director}'}}) \
match (t:titulo) \
match (d) -- (t) -- (a) \
return a;\
"
c1

"match (a:actor) match (d:director {nombre: 'Wallace Fox'}) match (t:titulo) match (d) -- (t) -- (a) return a;"

In [108]:
session.run(c1).data()

[{'a': {'id_persona': '64300', 'nombre': "Robert 'Buzz' Henry"}},
 {'a': {'id_persona': '127876', 'nombre': 'Stuart Erwin'}},
 {'a': {'id_persona': '27280', 'nombre': 'Eddie Kane'}},
 {'a': {'id_persona': '14166', 'nombre': 'Bess Flowers'}},
 {'a': {'id_persona': '564479', 'nombre': 'Marcy McGuire'}},
 {'a': {'id_persona': '570400', 'nombre': 'Renee Helms'}},
 {'a': {'id_persona': '108232', 'nombre': 'Charles Williams'}},
 {'a': {'id_persona': '45704', 'nombre': 'Charles Judels'}},
 {'a': {'id_persona': '47792', 'nombre': 'Gladys Blake'}},
 {'a': {'id_persona': '153889', 'nombre': 'Lorraine Krueger'}},
 {'a': {'id_persona': '570009', 'nombre': 'Ariel Heath'}},
 {'a': {'id_persona': '326540', 'nombre': 'Alex Craig'}},
 {'a': {'id_persona': '133960', 'nombre': 'Linda Brent'}},
 {'a': {'id_persona': '565316', 'nombre': 'Craig Woods'}},
 {'a': {'id_persona': '17301', 'nombre': 'Iris Adrian'}},
 {'a': {'id_persona': '127299', 'nombre': 'Edward Norris'}},
 {'a': {'id_persona': '49038', 'nomb

In [118]:
#Prueba funcion de consulta 1

def consulta1(ac):
    c0 = f"\
match (a:actor {{nombre: '{ac[0]}'}}) \
match (b:actor {{nombre: '{ac[1]}'}}) \
match (t:titulo) \
match (a) -- (t) -- (b) \
return (t)\
"

    a = session.run(c0).data()[0]["t"]["id_titulo"]
    #print(type(a))

    consulta = []
    for j in table.find({'id_pelicula':a}):
        consulta.append(j)

    df = pd.DataFrame.from_dict(consulta)
    df = df[["título","descripción","calificación_imdb"]]

    return df

In [123]:
result = consulta1(bc)
result

,título,descripción,calificación_imdb
0,"Come On, Rangers",A Texas Ranger (Roy Rogers) and his pals come ...,6.2
1,Wall Street Cowboy,"When his ranch falls on hard times, Cowboy Roy...",6.0
2,Frontier Pony Express,"In the midst of the Civil War, Lassiter has a ...",6.2
3,Rough Riders' Round-up,Roy Rogers is a cowboy who joins the Border Pa...,5.4


In [114]:
#Prueba función de consulta2

def consulta2(director,cantidad):
    c1 = f"\
match (a:actor) \
match (d:director {{nombre: '{director}'}}) \
match (t:titulo) \
match (d) -- (t) -- (a) \
return a;\
"

    a = session.run(c1).data()
    consulta = []
    for v in a:
        consulta.append(v["a"]["nombre"])
    return consulta[:cantidad]

consulta2(director, 20)

["Robert 'Buzz' Henry",
 'Stuart Erwin',
 'Eddie Kane',
 'Bess Flowers',
 'Marcy McGuire',
 'Renee Helms',
 'Charles Williams',
 'Charles Judels',
 'Gladys Blake',
 'Lorraine Krueger',
 'Ariel Heath',
 'Alex Craig',
 'Linda Brent',
 'Craig Woods',
 'Iris Adrian',
 'Edward Norris',
 'Frances Langford',
 'David Durand',
 'Al Stone',
 'Mary Gordon']

In [ ]:
#Dadas dos peliculas, encontrar a quienes actuaron en ambas:

match (a:actor)
match (p:movie {nombre:'_peliculaA_'})
match (k:movie {nombre:'_peliculaB_'})
match (p) -- (a) -- (k)
return (a)
;

In [104]:
pc = ["Come On, Rangers", "Wall Street Cowboy"]

c2 = f"\
match (a:actor) \
match (p:movie {{nombre:'{pc[0]}'}}) \
match (k:movie {{nombre:'{pc[1]}'}}) \
match (p) -- (a) -- (k) \
return (a) \
"
c2

"match (a:actor) match (p:movie {nombre:'Come On, Rangers'}) match (k:movie {nombre:'Wall Street Cowboy'}) match (p) -- (a) -- (k) return (a) "

In [115]:
session.run(c2).data()

[{'a': {'id_persona': '130905', 'nombre': 'Raymond Hatton'}},
 {'a': {'id_persona': '123795', 'nombre': 'Roy Rogers'}}]

In [122]:
##Prueba función de consulta3

def consulta3(pc,cantidad):
    c2 = f"\
match (a:actor) \
match (p:movie {{nombre:'{pc[0]}'}}) \
match (k:movie {{nombre:'{pc[1]}'}}) \
match (p) -- (a) -- (k) \
return (a) \
"
    a = session.run(c2).data()
    consulta = []
    for v in a:
        consulta.append(v["a"]["nombre"])
    return consulta[:cantidad]

bc = consulta3(pc, 20)